In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((224, 224)),  # Add random crop
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=45),  # Increase rotation angle
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.GaussianBlur(kernel_size=3),  # Add Gaussian blur
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)),  # Add random erasing
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the image
]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 28 * 28, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 102)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 256 * 28 * 28)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)


model = NeuralNetwork().to(device)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
epochs = 100
start = time.time()
best_val_loss = float('inf')
patience = 5
counter = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/100
Train Loss: 4.3264, Val Loss: 4.4291, Val Accuracy: 5.69
Epoch 2/100
Train Loss: 2.8640, Val Loss: 4.2986, Val Accuracy: 9.31
Epoch 3/100
Train Loss: 1.8076, Val Loss: 3.9718, Val Accuracy: 19.31
Epoch 4/100
Train Loss: 0.9795, Val Loss: 3.8239, Val Accuracy: 26.57
Epoch 5/100
Train Loss: 0.5296, Val Loss: 3.7587, Val Accuracy: 31.27
Epoch 6/100
Train Loss: 0.3132, Val Loss: 3.6796, Val Accuracy: 32.65
Epoch 7/100
Train Loss: 0.2188, Val Loss: 3.6707, Val Accuracy: 33.14
Epoch 8/100
Train Loss: 0.1677, Val Loss: 3.6577, Val Accuracy: 33.63
Epoch 9/100
Train Loss: 0.1378, Val Loss: 3.6637, Val Accuracy: 33.82
Epoch 10/100
Train Loss: 0.1164, Val Loss: 3.6567, Val Accuracy: 33.43
Epoch 11/100
Train Loss: 0.0997, Val Loss: 3.6528, Val Accuracy: 33.63
Epoch 12/100
Train Loss: 0.0898, Val Loss: 3.6516, Val Accuracy: 34.22
Epoch 13/100
Train Loss: 0.0783, Val Loss: 3.6594, Val Accuracy: 33.82
Epoch 14/100
